1. Librerias

In [ ]:
# ------------------------ Visualización ------------------------
import matplotlib.pyplot as plt                          # trazado básico con Matplotlib
import matplotlib.dates as mdates                        # formateo de fechas en ejes
from matplotlib.dates import DateFormatter               # formatos personalizados de fecha
import matplotlib.patches as mpatches                    # creación de parches / leyendas
from matplotlib.patches import Ellipse                   #create and draw elliptical shapes within a Matplotlib plot.
%matplotlib inline
import seaborn as sns                                    # visualizaciones estadísticas construidas sobre Matplotlib


# ------------------------ Manipulación de datos ------------------------
import pandas as pd                                      # estructuras tipo DataFrame
import numpy as np
from sklearn.preprocessing import StandardScaler         #Z-score normalization.
from sklearn.decomposition import PCA
#from ydata_profiling import ProfileReport                #analisis exploratorio
from scipy import stats
from scipy.cluster.hierarchy import linkage, dendrogram, set_link_color_palette

from scipy.cluster.hierarchy import dendrogram           #function is used to visualize the results of hierarchical clustering
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch
from sklearn.cluster import KMeans
from scipy.spatial import ConvexHull
from sklearn.metrics import silhouette_score

from sklearn.preprocessing import PolynomialFeatures   #transformación datos X al cuadrado

#----------------------- Factor Analyzer -------------------------------

'''
from factor_analyzer.factor_analyzer import calculate_kmo
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
from factor_analyzer import FactorAnalyzer
import networkx as nx
'''

#---------------------- linear regression (OLS), generalized linear models (GLM), and various statistical tests.
import statsmodels.api as sm
from statsmodels.stats.diagnostic import linear_reset
from statsmodels.stats.diagnostic import het_white
from statsmodels.sandbox.stats.runs import runstest_1samp
from scipy.stats import kstest
from sklearn.linear_model import LinearRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor

from statsmodels.stats.stattools import durbin_watson
#Cargar las librerías


from collections import Counter
import math


#---------------------- UC Irvine Machine Learning Repository ---------------------------------------------
from ucimlrepo import fetch_ucirepo


2. Cargar el conjunto de datos

In [ ]:
#codigo de https://archive.ics.uci.edu/dataset/555/apartment+for+rent+classified
idcode=555
   

try:
    from ucimlrepo import fetch_ucirepo
    # fetch dataset 
    apartment_for_rent_classified = fetch_ucirepo(id=idcode) 
       
    # data (as pandas dataframes) 
    datos = apartment_for_rent_classified.data.features
      

    print(apartment_for_rent_classified.variables) 


except ModuleNotFoundError:
    print("La librería 'ucimlrepo' no está instalada.")
    # Puedes añadir aquí un comando para instalarla, por ejemplo:
    import subprocess
    subprocess.check_call(["pip", "install", "ucimlrepo"])

except Exception as e:  # Catching other types of exceptions
    print(f"Error inesperado: {e}")
    

EDA

In [10]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99826 entries, 0 to 99825
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   category       99826 non-null  object 
 1   title          99826 non-null  object 
 2   body           99826 non-null  object 
 3   amenities      83749 non-null  object 
 4   bathrooms      99760 non-null  object 
 5   bedrooms       99699 non-null  object 
 6   currency       99822 non-null  object 
 7   fee            99823 non-null  object 
 8   has_photo      99823 non-null  object 
 9   pets_allowed   39192 non-null  object 
 10  price          99821 non-null  float64
 11  price_display  99820 non-null  object 
 12  price_type     99823 non-null  object 
 13  square_feet    99823 non-null  object 
 14  address        7946 non-null   object 
 15  cityname       99521 non-null  object 
 16  state          99521 non-null  object 
 17  latitude       99797 non-null  float64
 18  longit

In [ ]:
datos.describe()

In [ ]:
datos.head()

##Limpieza de datos

In [ ]:

#datos.loc[:,'bathrooms'] = pd.to_numeric(datos['bathrooms'], errors='coerce')
#datos.loc[:,'bedrooms'] = pd.to_numeric(datos['bedrooms'], errors='coerce')
#datos.loc[:,'square_feet'] = pd.to_numeric(datos['square_feet'], errors='coerce')
datostransformados = datos.copy()

datostransformados['bathrooms'] = pd.to_numeric(datos['bathrooms'], errors='coerce')
datostransformados['bedrooms'] = pd.to_numeric(datos['bedrooms'], errors='coerce')
datostransformados['square_feet'] = pd.to_numeric(datos['square_feet'], errors='coerce')


#has_foto a binario, es irreelante si tiene foto o es thumbnail:
mapeo_foto = {'Yes': 1, 'Thumbnail': 1, 'No': 0}
datostransformados['has_photo_bin'] = datos['has_photo'].map(mapeo_foto)
del datostransformados['has_photo']

mapeo_fee = {'Yes': 1,'No': 0}
datostransformados['has_fee_bin'] = datos['fee'].map(mapeo_fee)
del datostransformados['fee']

#onehot encoding

datostransformados = pd.get_dummies(datostransformados, columns=['category'], prefix='cat', drop_first=True, dtype=int)
#datostransformados = pd.get_dummies(datostransformados, columns=['category'], prefix='cat', drop_first=True)


datostransformados['pets_allowed'].fillna('No',inplace=True)
datostransformados = pd.get_dummies(datostransformados, columns=['pets_allowed'], prefix='pets', drop_first=True, dtype=int)


datostransformados = pd.get_dummies(datostransformados, columns=['price_type'], prefix='pri', drop_first=True, dtype=int)

#error al leer los datos, borrar columnas que no deberian estar creadas, posiblemente por comas en las columnas title o body\

del datostransformados['cat_Gym']
del datostransformados['pets_Monthly']
del datostransformados['pri_Los Angeles']
del datostransformados['pri_VA']
                       


In [ ]:
datostransformados.info()

In [12]:
#dataset con datos borrados
datosclean=datostransformados[['bathrooms', 'bedrooms', 'price','square_feet','latitude','longitude','time','has_photo_bin',
                              'has_fee_bin','cat_Parking,Patio/Deck','cat_housing/rent','cat_housing/rent/apartment',
                              'cat_housing/rent/commercial/retail','cat_housing/rent/condo','cat_housing/rent/home',
                              'cat_housing/rent/other','cat_housing/rent/short_term','pets_Cats','pets_Cats,Dogs',
                              'pets_Cats,Dogs,None','pets_Dogs','pets_No','pri_Monthly','pri_Monthly|Weekly','pri_Weekly'
]].copy()
datosclean.dropna(inplace=True)
datosclean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 99608 entries, 0 to 99825
Data columns (total 25 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   bathrooms                           99608 non-null  float64
 1   bedrooms                            99608 non-null  float64
 2   price                               99608 non-null  float64
 3   square_feet                         99608 non-null  float64
 4   latitude                            99608 non-null  float64
 5   longitude                           99608 non-null  float64
 6   time                                99608 non-null  float64
 7   has_photo_bin                       99608 non-null  float64
 8   has_fee_bin                         99608 non-null  float64
 9   cat_Parking,Patio/Deck              99608 non-null  int64  
 10  cat_housing/rent                    99608 non-null  int64  
 11  cat_housing/rent/apartment          99608 non-

In [13]:
datosclean.to_csv('dataset.csv', sep='|')